In [5]:
# 因果推断
# 竞争站点价差、或价格，或价格变化是否对我方整体销量/石油/柴油构成影响


In [7]:
### STEP1：读取数据 ###

import pandas as pd
from dowhy import CausalModel
from IPython.display import Image, display

COLS_DATE = 'date'
COLS_PRICE = '60000359_price'                   #92号国六B
COLS_PRICE_OTHER = '60000359_price_other'
COLS_AMOUNT = '60000359_amount'  
COLS_CAR_FLOW = '进站加油车量（柴油）'
# COLS_CAR_FLOW = '进站加油车量（汽油）'
COLS_ALL = [COLS_DATE, 
            COLS_CAR_FLOW,
            COLS_PRICE, COLS_PRICE_OTHER, COLS_AMOUNT
            ]


# 读取excel文件并获取指定列
df = pd.read_csv('../Data/竞争站分析-竞争站-单站-32801817.csv', usecols=COLS_ALL, encoding='GBK')
print(df.head())


       date  进站加油车量（柴油）  60000359_price 60000359_price_other  60000359_amount
0  2023/1/1          12            7.20         [5.07, 7.20]        186516.28
1  2023/1/2          22            7.20         [5.07, 7.20]        247329.50
2  2023/1/3          13            7.20         [5.07, 7.20]        314626.46
3  2023/1/4           7            7.41   [5.07, 7.21, 7.41]         18120.33
4  2023/1/5          19            7.41         [5.07, 7.41]         98104.99


In [ ]:
### STEP2：特征处理 ###

In [ ]:
### STEP3：

# 假设 dataset 是包含吸烟和肺癌数据的 DataFrame
causal_graph = dowhy.CausalGraph(data=dataset)
causal_graph.add_variable(parent_list=["Smoking"], child="LungCancer")